# Axion - Stock Recommendation System
Factor-based stock selection with backtest analysis.

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

import config
from src.universe import build_universe
from src.data_fetcher import download_price_data, download_fundamentals, compute_price_returns, filter_universe
from src.factor_model import compute_composite_scores
from src.portfolio import select_top_stocks, compute_allocations
from src.backtest import run_backtest

## 1. Universe Exploration

In [ ]:
tickers = build_universe(verbose=True)
print(f"\nTotal universe: {len(tickers)} tickers")

In [ ]:
# Download data
prices = download_price_data(tickers, verbose=True)
fundamentals = download_fundamentals(tickers, verbose=True)
print(f"\nPrices shape: {prices.shape}")
print(f"Fundamentals shape: {fundamentals.shape}")

In [ ]:
# Filter universe
valid_tickers = filter_universe(fundamentals, prices, verbose=True)
fund = fundamentals.loc[fundamentals.index.isin(valid_tickers)]
px = prices[[c for c in prices.columns if c in valid_tickers]]
print(f"Filtered: {len(valid_tickers)} tickers")

## 2. Factor Distributions

In [ ]:
returns = compute_price_returns(px)
scores = compute_composite_scores(fund, returns, verbose=True)
scores = scores.loc[scores.index.isin(valid_tickers)]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
for ax, col in zip(axes.flat, ["value", "momentum", "quality", "growth", "composite"]):
    scores[col].hist(ax=ax, bins=40, alpha=0.7)
    ax.set_title(col.capitalize())
    ax.axvline(scores[col].median(), color="red", linestyle="--", alpha=0.7)
axes.flat[-1].set_visible(False)
plt.tight_layout()
plt.show()

In [ ]:
# Factor correlation matrix
corr = scores[["value", "momentum", "quality", "growth"]].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="RdBu_r", center=0, vmin=-1, vmax=1)
plt.title("Factor Correlation Matrix")
plt.tight_layout()
plt.show()

## 3. Portfolio Construction

In [ ]:
AMOUNT = 10000

top_stocks = select_top_stocks(scores, verbose=True)
portfolio = compute_allocations(top_stocks, fund, AMOUNT, verbose=True)
portfolio

In [ ]:
# Portfolio allocation pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

ax1.pie(portfolio["weight"], labels=portfolio["ticker"], autopct="%1.1f%%", startangle=90)
ax1.set_title("Portfolio Weights")

ax2.barh(portfolio["ticker"], portfolio["score"], color="steelblue")
ax2.set_xlabel("Composite Score")
ax2.set_title("Factor Scores")
plt.tight_layout()
plt.show()

## 4. Backtest

In [ ]:
results = run_backtest(px, fund, verbose=True)

if "error" not in results:
    metrics = results["metrics"]
    for k, v in metrics.items():
        if isinstance(v, float):
            print(f"{k:25s}: {v:.4f}")
        else:
            print(f"{k:25s}: {v}")

In [ ]:
# Cumulative returns plot
if "error" not in results:
    port_cum = np.cumprod(1 + results["portfolio_returns"])
    bench_cum = np.cumprod(1 + results["benchmark_returns"])

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    ax1.plot(port_cum, label="Portfolio", linewidth=2)
    ax1.plot(bench_cum, label="SPY Benchmark", linewidth=2, alpha=0.7)
    ax1.set_ylabel("Cumulative Return")
    ax1.set_title("Backtest: Cumulative Returns")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Drawdown
    running_max = np.maximum.accumulate(port_cum)
    drawdown = port_cum / running_max - 1
    ax2.fill_between(range(len(drawdown)), drawdown, alpha=0.4, color="red")
    ax2.set_ylabel("Drawdown")
    ax2.set_xlabel("Month")
    ax2.set_title("Portfolio Drawdown")
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

## 5. Sensitivity Analysis

In [ ]:
# Test different factor weight configurations
weight_configs = {
    "Balanced": {"value": 0.25, "momentum": 0.25, "quality": 0.25, "growth": 0.25},
    "Momentum Heavy": {"value": 0.15, "momentum": 0.45, "quality": 0.20, "growth": 0.20},
    "Value Heavy": {"value": 0.45, "momentum": 0.20, "quality": 0.20, "growth": 0.15},
    "Quality Heavy": {"value": 0.20, "momentum": 0.20, "quality": 0.45, "growth": 0.15},
    "Default": config.FACTOR_WEIGHTS,
}

sensitivity_results = []
for name, weights in weight_configs.items():
    composite = (
        weights["value"] * scores["value"]
        + weights["momentum"] * scores["momentum"]
        + weights["quality"] * scores["quality"]
        + weights["growth"] * scores["growth"]
    )
    top = composite.nlargest(config.TOP_N_STOCKS)
    sensitivity_results.append({
        "config": name,
        "avg_score": top.mean(),
        "min_score": top.min(),
        "tickers": list(top.index),
    })

sens_df = pd.DataFrame(sensitivity_results)
print(sens_df[["config", "avg_score", "min_score"]].to_string(index=False))
print("\nSelected tickers by configuration:")
for _, row in sens_df.iterrows():
    print(f"  {row['config']:20s}: {', '.join(row['tickers'])}")